<a href="https://colab.research.google.com/github/pooyapaydary/Python/blob/main/micro_project_01Nov25.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#import everything
import pandas as pd
import re
import numpy as np
import sqlite3

In [98]:
#read the data
wdf = pd.read_csv("water_quality_readings.csv")
sdf = pd.read_csv("stations.csv")

#look at the data
#print(wdf.head())
#print("\n")
#print(sdf.head(5))

#print(wdf.info())
#print("\n")
#print(sdf.info())

#cleaning the data
  #1-change sample time to date-time
  #2-trim string with function
     #define function
def clean_up(c):

  if type(c)==str:
      c = c.lower().strip()  #lower and strip spaces
      c= re.sub(r'[^A-Za-z0-9_ ]+', "", c)  #remove weird charchters
      if c =="nan":
        return (None)
  return(c)

#apply the cleaning function to the dataframe
for col  in wdf.columns :
  wdf[col]=wdf[col].apply(clean_up)

for col  in sdf.columns :
  sdf[col]=sdf[col].apply(clean_up)


#change date time to date time
wdf["sample_time"] = pd.to_datetime(wdf["sample_time"], errors="coerce")

#impute missing pH values with station-wise median

#I learned this code here. "transform"
wdf["pH"] = wdf["pH"].fillna(wdf.groupby("station_id")["pH"].transform("median"))

#lets do it for Turbidity
wdf["turbidity_NTU"]=wdf["turbidity_NTU"].fillna(wdf.groupby("station_id")["turbidity_NTU"].transform("median"))


#lets write a function to do that to all columns in wdf
def med (c):
  wdf[c]=wdf[c].fillna(wdf.groupby("station_id")[c].transform("median"))

#apply the function in a for loop
for c in ['pH', 'turbidity_NTU',
       'temperature_C', 'dissolved_oxygen_mgL', 'nitrate_mgL', 'lead_ugL']:
  med(c)

#flag high lead and turb
wdf["high_lead"]=wdf["lead_ugL"]>15
wdf["high_turb"]=wdf["turbidity_NTU"]>5


#now extract month and year
wdf["month"]=wdf["sample_time"].dt.to_period("M")

#now extract if that day was weekend
wdf["is_weekend"]=wdf["sample_time"].dt.day_of_week>5

#lets see if it was day-time
wdf["is_daytime"]=wdf["sample_time"].dt.hour.between(9,17)

#now lets merge with stations

df=wdf.merge(sdf, how="left", on="station_id")
#print(df.info())

df["station_id"].unique()
#print(wdf.head())


for c in ["pH", "turbidity_NTU", "temperature_C", "dissolved_oxygen_mgL", "nitrate_mgL", "lead_ugL", "e_coli_CFU_100mL"]:
  a=df.groupby("month")[c].mean()
  print(type(a))



<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
<class 'pandas.core.series.Series'>
